# Implement Quantum Teleportation algorithm in Python

In [ ]:
# python -m venv qiskit_env
 
#  qiskit_env\Scripts\activate  

# pip install qiskit 
# pip install jupyter
# pip install qiskit-aer  
# pip install matplotlib 
# pip install qiskit-ibm-runtime

# Import necessary libraries from Qiskit

# QuantumCircuit: Allows us to define and manipulate quantum circuits.
# transpile: Optimizes the circuit for specific backends.
# QuantumRegister, ClassicalRegister: Used to define quantum and classical registers.
# Aer: Provides access to the Qiskit Aer simulator for running quantum circuits locally.
# plot_histogram: Visualizes the measurement results in a histogram.
# QiskitRuntimeService: Connects to IBM's quantum service for accessing real quantum devices.
# Sampler, Estimator: (Not used here directly) used for sampling and estimating circuit results on IBM hardware.
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService

# Step 1: Initialize IBM Quantum Service
# --------------------------------------
# QiskitRuntimeService connects to IBM Quantum for running the circuit on real quantum hardware (optional).
# Here, we're specifying the IBM Quantum service.
service = QiskitRuntimeService(channel="ibm_quantum",token='')

# Step 2: Create Quantum and Classical Registers
# ----------------------------------------------
# QuantumRegister: Defines 3 qubits for the teleportation protocol.
# A Quantum Register in Qiskit is a data structure that holds a specified number of qubits 
# for use in a quantum circuit. It acts like a container for qubits, 
# where each qubit can store quantum information and interact with other qubits through quantum gates.


# ClassicalRegister: Defines 3 classical bits to store measurement results.
# Classical Register is used to store the results of measurements made on qubits in a quantum circuit. 
# When you perform measurements on qubits, the quantum information (which is in superposition) collapses to 
# classical values (0 or 1), and those values are stored in classical registers. 
# Classical registers are essential for reading the final state of the quantum system.

q = QuantumRegister(3, 'q')               # Quantum register with 3 qubits
# q = QuantumRegister(3, 'q'), we are creating a quantum register named q that contains 3 qubits. 
# q[0], q[1], q[2]: These are the individual qubits within the register q,
#  and we can apply operations (like gates) to each one individually or collectively.

c0 = ClassicalRegister(1, 'c0')           # Classical register for measurement of qubit 0
c1 = ClassicalRegister(1, 'c1')           # Classical register for measurement of qubit 1
c2 = ClassicalRegister(1, 'c2')           # Classical register for measurement of qubit 2
# ClassicalRegister(1, 'c0'): Creates a classical register named 'c0' with 1 classical bit, 
# which will store the measurement result of qubit 0. same for others

circuit = QuantumCircuit(q, c0, c1, c2)   # Define the quantum circuit with these registers

# Step 3: Initialize the Qubit to Teleport
# ----------------------------------------
# We want to teleport the quantum state [0, 1] (|1⟩ state) of qubit 0.
# Initialize the first qubit in state |1⟩, which will be teleported to the third qubit.
circuit.initialize([0, 1], q[0])          # Initialize qubit 0 in state |1⟩
circuit.barrier()                         # Barrier for visual separation in the circuit

# Step 4: Create an Entangled Pair (Bell State) Between Qubits 1 and 2
# ---------------------------------------------------------------------
# Apply Hadamard and CNOT gates to create entanglement between qubits 1 and 2.
circuit.h(q[1])                           # Apply Hadamard gate to qubit 1
circuit.cx(q[1], q[2])                    # Apply CNOT gate with qubit 1 as control and qubit 2 as target
circuit.barrier()                         # Barrier for visual separation in the circuit

# Step 5: Perform Bell Measurement on Qubits 0 and 1
# ---------------------------------------------------
# We entangle qubit 0 with qubit 1 and measure them to enable teleportation.
circuit.cx(q[0], q[1])                    # Apply CNOT gate with qubit 0 as control and qubit 1 as target
circuit.h(q[0])                           # Apply Hadamard gate to qubit 0
circuit.barrier()                         # Barrier for visual separation in the circuit

# Step 6: Measure Qubits 0 and 1
# ------------------------------
# Measure qubits 0 and 1 to record the state for use in conditional operations on qubit 2.
circuit.measure(q[0], c0[0])              # Measure qubit 0 and store the result in classical register c0
circuit.measure(q[1], c1[0])              # Measure qubit 1 and store the result in classical register c1

# Step 7: Apply Conditional Gates on Qubit 2 Based on Measurement Results
# -----------------------------------------------------------------------
# The results in c0 and c1 will determine the correction gates (X and Z) on qubit 2.
circuit.x(q[2]).c_if(c1, 1)               # Apply X gate on qubit 2 if measurement on qubit 1 (c1) is 1
circuit.z(q[2]).c_if(c0, 1)               # Apply Z gate on qubit 2 if measurement on qubit 0 (c0) is 1

# Step 8: Measure the Teleported Qubit's State (Qubit 2)
# ------------------------------------------------------
# Measure qubit 2 to verify if it has the original state of qubit 0.
circuit.measure(q[2], c2[0])              # Measure qubit 2 and store the result in classical register c2

# Print the circuit diagram to visualize the teleportation steps
print(circuit)

# Draw the circuit
circuit_drawer(circuit)


# Step 9: Run the Simulation
# --------------------------
# Use the Aer simulator to execute the circuit and check if qubit 2 matches the initial state of qubit 0.
simulator = Aer.get_backend('qasm_simulator')      # Use the 'qasm_simulator' backend
job = simulator.run(circuit, shots=1)              # Run the job with 1 shot to check for teleportation
result = job.result()                              # Get the result of the execution
teleported_state = result.get_counts(circuit)      # Get the measurement counts

# Display the results
print('Teleported state:', teleported_state)       # Print the teleported state
plot_histogram(teleported_state)                   # Plot the histogram to visualize results


# q[1] and q[2] are entangled to create a Bell state (an entangled pair).
# The entanglement is achieved by applying a Hadamard gate to q[1] followed by
#  a CNOT gate between q[1] (control) and q[2] (target).

# Qubits 1 and 2 (q[1] and q[2]) are entangled.
# Alice sends information about the state of q[0] (her qubit) to Bob via classical communication.
# Based on Alice's measurements (on q[0] and q[1]), Bob performs conditional operations (X and Z gates) 
# on his qubit (q[2]) to reconstruct the state of q[0].
